<a href="https://colab.research.google.com/github/Anuj2409/Loan-Fraud-Detection-problem/blob/master/XGB_NN_Home_Credit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> **HOME CREDIT FRAUD DETECTION**


In this project, we take home credit defaulters data and try to predict the defaulters accurately using different machine learning algorithms. The data which I use here is highly imbalanced data with about only 9% defaulters. This makes difficult for the machine to identify the patterns and hence it is really difficult to get very high accuracy for such classifications.

There are 6 data files in total having different sort of loan application details like current financial information, demographic information, previous application information and so on. When we combine these data tables we get around 483 attributes for each application. But I saw that there were many columns which had upto 99.6% missing data which makes the analysis difficult. So we remove all the attributes which have more than 30% missing values.

Now that I had a smaller subset of the data, I ran XG Boost model to obtain most important features. This gave me 72 features which are used for creating the models.

In this notebook, I have focused on comparing the performance of Logistic Regression with a XGB model and a basic neural network classifier.



> **Loading libraries**



In [0]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.datasets import load_iris
from sklearn import tree
from sklearn.model_selection import cross_validate, KFold
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, roc_curve,auc
import matplotlib.pyplot as plt
from sklearn.svm import SVR
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LogisticRegression
pd.set_option('display.max_columns', None)



> **Loading the data from google drive**
* I have mounted my drive here before pulling the data from the drive



In [0]:
masterdf0 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Master_Data_with_filtering_updated.csv')
masterdf_test0 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Test_Master_Data_with_filtering_updated.csv')

I decided to take the variables shown in the following chunk of code because this is what I got from XGBoost variable importance

In [0]:
 selected_features = ['SK_ID_CURR','TARGET','EXT_SOURCE_3'
 ,'EXT_SOURCE_2'
 ,'NAME_CONTRACT_STATUS_Refused'
 ,'NAME_EDUCATION_TYPE_Higher education'
 ,'AMT_GOODS_PRICE_x'
 ,'NAME_INCOME_TYPE_Working'
 ,'NAME_INCOME_TYPE_Pensioner'
 ,'FLAG_DOCUMENT_3'
 ,'CODE_GENDER_F'
 ,'AMT_DOWN_PAYMENT'
 ,'DAYS_BIRTH'
 ,'MONTHS_BALANCE_MIN'
 ,'FLAG_OWN_CAR_N'
 ,'AMT_GOODS_PRICE_y'
 ,'NAME_PORTFOLIO_Cards'
 ,'NAME_YIELD_GROUP_high'
 ,'DAYS_EMPLOYED'
 ,'Active'
 ,'Revolving loans'
 ,'AVG_AMT_PAYMENT'
 ,'Consumer loans'
 ,'NAME_EDUCATION_TYPE_Secondary / secondary special'
 ,'AMT_ANNUITY_x'
 ,'AMT_CREDIT_x'
 ,'CODE_GENDER_M'
 ,'PREVIOUS_REJECTS'
 ,'DEF_30_CNT_SOCIAL_CIRCLE'
 ,'SK_ID_PREV'
 ,'DAYS_LAST_PHONE_CHANGE'
 ,'AMT_CREDIT_SUM_DEBT'
 ,'CNT_INSTALMENT'
 ,'REG_CITY_NOT_LIVE_CITY'
 ,'CREDIT_ACTIVE_C'
 ,'CNT_PAYMENT'
 ,'REGION_RATING_CLIENT_W_CITY'
 ,'AMT_CREDIT_SUM_OVERDUE'
 ,'DAYS_LAST_DUE_1ST_VERSION'
 ,'AVG_DAYS_ENTRY_PAYMENT'
 ,'DAYS_ID_PUBLISH'
 ,'CNT_INSTALMENT_FUTURE'
 ,'MONTHS_BALANCE'
 ,'DAYS_FIRST_DUE'
 ,'AMT_CREDIT_SUM'
 ,'DAYS_LAST_DUE'
 ,'NAME_PORTFOLIO_POS'
 ,'ID_PREV_NUM'
 ,'NAME_YIELD_GROUP_low_action'
 ,'SELLERPLACE_AREA'
 ,'AMT_INCOME_TOTAL'
 ,'DAYS_DECISION'
 ,'ORGANIZATION_TYPE_Self-employed'
 ,'CREDIT_ACTIVE_A'
 ,'NAME_CONTRACT_TYPE_Cash loans'
 ,'NAME_FAMILY_STATUS_Married'
 ,'AMT_APPLICATION'
 ,'AMT_ANNUITY_y.1'
 ,'STATUS_C'
 ,'NAME_YIELD_GROUP_middle'
 ,'DEF_60_CNT_SOCIAL_CIRCLE'
 ,'OBS_60_CNT_SOCIAL_CIRCLE'
 ,'AMT_CREDIT_MAX_OVERDUE'
 ,'AVG_DAYS_INSTALMENT'
 ,'REGION_POPULATION_RELATIVE'
 ,'NAME_EDUCATION_TYPE_Lower secondary'
 ,'STATUS_1'
 ,'CNT_INSTALMENT_FUTURE_MIN'
 ,'HOUR_APPR_PROCESS_START'
 ,'AMT_REQ_CREDIT_BUREAU_YEAR'
 ,'AMT_CREDIT_y'
 ,'WEEKDAY_APPR_PROCESS_START_TUESDAY'
 ,'Completed'
 ,'AVG_AMT_INSTALMENT']

 masterdf = masterdf0[selected_features]
 selected_features.pop(1)
 masterdf_test = masterdf_test0[selected_features]

In [0]:
masterdf.shape

(307511, 74)

In [0]:
masterdf_test.shape

(48744, 73)

One particular row has infinite income. Hence, I removed that application from the analysis.

In [0]:
masterdf = masterdf[masterdf['SK_ID_CURR'] != 333092]

In [0]:
masterdf_test.describe()

,SK_ID_CURR,EXT_SOURCE_3,EXT_SOURCE_2,NAME_CONTRACT_STATUS_Refused,NAME_EDUCATION_TYPE_Higher education,AMT_GOODS_PRICE_x,NAME_INCOME_TYPE_Working,NAME_INCOME_TYPE_Pensioner,FLAG_DOCUMENT_3,CODE_GENDER_F,AMT_DOWN_PAYMENT,DAYS_BIRTH,MONTHS_BALANCE_MIN,FLAG_OWN_CAR_N,AMT_GOODS_PRICE_y,NAME_PORTFOLIO_Cards,NAME_YIELD_GROUP_high,DAYS_EMPLOYED,Active,Revolving loans,AVG_AMT_PAYMENT,Consumer loans,NAME_EDUCATION_TYPE_Secondary / secondary special,AMT_ANNUITY_x,AMT_CREDIT_x,CODE_GENDER_M,PREVIOUS_REJECTS,DEF_30_CNT_SOCIAL_CIRCLE,SK_ID_PREV,DAYS_LAST_PHONE_CHANGE,AMT_CREDIT_SUM_DEBT,CNT_INSTALMENT,REG_CITY_NOT_LIVE_CITY,CREDIT_ACTIVE_C,CNT_PAYMENT,REGION_RATING_CLIENT_W_CITY,AMT_CREDIT_SUM_OVERDUE,DAYS_LAST_DUE_1ST_VERSION,AVG_DAYS_ENTRY_PAYMENT,DAYS_ID_PUBLISH,CNT_INSTALMENT_FUTURE,MONTHS_BALANCE,DAYS_FIRST_DUE,AMT_CREDIT_SUM,DAYS_LAST_DUE,NAME_PORTFOLIO_POS,ID_PREV_NUM,NAME_YIELD_GROUP_low_action,SELLERPLACE_AREA,AMT_INCOME_TOTAL,DAYS_DECISION,ORGANIZATION_TYPE_Self-employed,CREDIT_ACTIVE_A,NAME_CONTRACT_TYPE_Cash loans,NAME_FAMILY_STATUS_Married,AMT_APPLICATION,AMT_ANNUITY_y.1,STATUS_C,NAME_YIELD_GROUP_middle,DEF_60_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,AMT_CREDIT_MAX_OVERDUE,AVG_DAYS_INSTALMENT,REGION_POPULATION_RELATIVE,NAME_EDUCATION_TYPE_Lower secondary,STATUS_1,CNT_INSTALMENT_FUTURE_MIN,HOUR_APPR_PROCESS_START,AMT_REQ_CREDIT_BUREAU_YEAR,AMT_CREDIT_y,WEEKDAY_APPR_PROCESS_START_TUESDAY,Completed,AVG_AMT_INSTALMENT
count,48744.000000,40076.000000,48736.000000,48744.000000,48744.000000,4.874400e+04,48744.000000,48744.000000,48744.000000,48744.000000,4.874400e+04,48744.000000,47808.000000,48744.000000,47711.000000,48744.000000,48744.000000,48744.000000,47808.000000,47800.000000,47943.000000,47800.000000,48744.000000,48720.000000,4.874400e+04,48744.000000,47800.000000,48715.000000,47808.000000,48744.000000,4.228200e+04,47804.000000,48744.000000,42282.000000,47737.000000,48744.000000,4.228200e+04,47580.000000,47943.000000,48744.000000,47804.000000,47808.000000,47580.000000,4.228200e+04,47580.000000,48744.000000,47944.000000,48744.000000,47800.000000,4.874400e+04,47800.000000,48744.000000,42282.000000,48744.000000,48744.000000,47775.000000,47737.000000,42282.000000,48744.000000,48715.000000,48715.000000,42282.000000,47944.000000,48744.000000,48744.000000,42282.000000,47804.000000,48744.000000,42695.000000,4.780000e+04,48744.000000,47808.000000,47944.000000
mean,277796.676350,0.500106,0.518021,0.052027,0.256770,4.626188e+05,0.503303,0.190239,0.786620,0.670400,9.017133e+03,-16068.084605,-53.909680,0.662871,100.087369,0.097530,0.342791,67485.366322,27.872783,0.665188,20521.495938,2.173536,0.697276,29426.240209,5.167404e+05,0.329600,1.024121,0.143652,30.496632,-1077.766228,7.370926e+05,20.424944,0.077466,3.820254,14.549004,2.012596,1.491646e+02,32489.918104,-928.995532,-3051.712949,20.308175,-12.778656,13537.960295,2.222549e+06,78525.490985,0.571127,3.012014,0.090042,412.465520,1.784318e+05,-917.552281,0.121451,2.096661,0.990994,0.662297,97.429441,15410.840530,107.081973,0.280527,0.101139,1.435738,0.334729,-917.315481,0.021226,0.009745,1.760513,0.417413,12.007365,1.983769,1.719261e+05,0.200045,2.301100,19973.251539
std,103169.547296,0.189498,0.181278,0.222084,0.436856,3.367102e+05,0.499994,0.392494,0.409698,0.470073,2.083791e+04,4325.900393,30.148924,0.472734,14.312588,0.296681,0.474647,144348.507136,23.949533,1.107042,25933.206176,1.872098,0.459442,16016.368315,3.653970e+05,0.470073,1.979746,0.514413,25.850590,878.920740,1.680264e+06,13.663026,0.267332,3.707577,8.342980,0.515804,9.211520e+03,57816.783827,598.098073,1569.276709,13.713268,18.561101,41757.175969,4.539563e+06,102782.486005,0.494920,2.111285,0.286245,2409.429125,1.015226e+05,555.368217,0.326654,1.815826,0.094474,0.472932,16.850828,10573.344603,126.602440,0.449261,0.403791,3.580125,0.722379,599.201814,0.014428,0.098235,4.405363,2.182407,3.278172,1.838873,1.602918e+05,0.400038,3.338029,24179.897932
min,100001.000000,0.000527,0.000008,0.000000,0.000000,4.500000e+04,0.000000



> Imputing the missing data



In [0]:
for col in masterdf.columns:
    masterdf[col] = masterdf[col].fillna(masterdf[col].median())

In [0]:
for col in masterdf_test.columns:
    masterdf_test[col] = masterdf_test[col].fillna(masterdf_test[col].median())

In [0]:
a = pd.DataFrame(masterdf.isna().sum())
a[a[0] > 0].sort_values(by = 0, ascending = False)

,0


In [0]:
a = pd.DataFrame(masterdf_test.isna().sum())
a[a[0] > 0].sort_values(by = 0, ascending = False)

,0




> Feature engineering

After exploring the data, I felt that there was a need of defining more variables to describe the loan application. Hence, I created 4 additional variables.

1.   Credit Income percentage - The ratio of credit amount to total income
2.   Annuity Income percentage - The ratio of annuity amount to total income
3.   Credit term - The term for the credit amount. Ratio of annuity amount to credit amount
4.   Percentage of days employeed - Percentage of days applicant is employeed in his life. Ratio of days employeed to days from his birth





In [0]:
masterdf['CREDIT_INCOME_PERCENT'] = masterdf['AMT_CREDIT_x'] /masterdf['AMT_INCOME_TOTAL']
masterdf['ANNUITY_INCOME_PERCENT'] = masterdf['AMT_ANNUITY_x'] /masterdf['AMT_INCOME_TOTAL']
masterdf['CREDIT_TERM'] = masterdf['AMT_ANNUITY_x'] /masterdf['AMT_CREDIT_x']
masterdf['DAYS_EMPLOYED_PERCENT'] = masterdf['DAYS_EMPLOYED'] /masterdf['DAYS_BIRTH']

In [0]:
masterdf_test['CREDIT_INCOME_PERCENT'] =masterdf_test['AMT_CREDIT_x'] /masterdf_test['AMT_INCOME_TOTAL']
masterdf_test['ANNUITY_INCOME_PERCENT'] =masterdf_test['AMT_ANNUITY_x'] /masterdf_test['AMT_INCOME_TOTAL']
masterdf_test['CREDIT_TERM'] =masterdf_test['AMT_ANNUITY_x'] /masterdf_test['AMT_CREDIT_x']
masterdf_test['DAYS_EMPLOYED_PERCENT'] =masterdf_test['DAYS_EMPLOYED'] /masterdf_test['DAYS_BIRTH']

In [0]:
masterdf_test = masterdf_test.set_index('SK_ID_CURR')

In [0]:
masterdf1 = masterdf[masterdf['TARGET'] == 1]
masterdf0 = masterdf[masterdf['TARGET'] == 0]

In [0]:
masterdf01, masterdf02 = train_test_split(masterdf0, test_size=0.2,random_state=45)

In [0]:
masterdf_ds = pd.concat([masterdf01, masterdf1], axis = 0)

In [0]:
X = masterdf_ds.iloc[:, 2:354]
y = masterdf_ds.iloc[:, 1:2]

In [0]:
masterdf_test.shape

(48744, 76)



> Split the data into test and train for the analysis



In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,random_state=45)



> Normalize data using standard scaler



In [0]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train_norm = scaler.transform(X_train)
X_test_norm = scaler.transform(X_test)
masterdf_test_norm = scaler.transform(masterdf_test)



> Changing the dimensions of the attributes to deal with the correlation within attributes retaining the maximum information.



In [0]:
from sklearn.decomposition import PCA
pca = PCA(n_components=76)
pca.fit(X_train_norm)
X_train_red = pca.transform(X_train_norm)
X_test_red = pca.transform(X_test_norm)
masterdf_test_red = pca.transform(masterdf_test_norm)

In [0]:
X_train_red = pd.DataFrame(X_train_red)
X_test_red = pd.DataFrame(X_test_red)
masterdf_test_red = pd.DataFrame(masterdf_test_red)

Logistic Regression

In [0]:
param_lr = dict(C= list((1e-4, 1e-3, 1e-2, 1e-1, 1)), penalty= ['l1', 'l2'])

lr_score = []
i = 42
lr = LogisticRegression(class_weight = {1:5, 0:1})

grid_lr = RandomizedSearchCV(lr, param_lr, cv = KFold(n_splits=3, shuffle=True, random_state=i), scoring = 'roc_auc')

grid_lr.fit(X_train_red, y_train)
lr_score.append(grid_lr.best_score_)

In [0]:
y_pred = grid_lr.predict(X_test_red)
y_pred2 = grid_lr.predict_proba(X_test_red)

In [0]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.91      0.93     56561
           1       0.25      0.33      0.29      4941

   micro avg       0.87      0.87      0.87     61502
   macro avg       0.59      0.62      0.61     61502
weighted avg       0.88      0.87      0.87     61502



In [0]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred2)

0.7436493359571911

XGBOOST

In [0]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import GridSearchCV

params = {'n_estimators':[250,500,750],'learning_rate':[0.001,0.01,0.1],'max_depth':[1,2,3,4,5,6]}

xgb2 = XGBClassifier(objective='binary:logistic',random_state=42,tree_method='gpu_hist')
clf = GridSearchCV(xgb2,params,cv=3,scoring='roc_auc')
clf.fit(X_train_red, y_train)

In [0]:
clf.best_params_

In [0]:
clf.best_score_

0.7603067415531021

In [0]:
auc2 = roc_auc_score(y_test, clf.predict(X_test_red))
print("xgb2 auc: %.4f" % auc2)

xgb2 auc: 0.5136


In [0]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import GridSearchCV

In [0]:
xgb_tuned = XGBClassifier(objective='binary:logistic',random_state=42,tree_method='gpu_hist',n_estimators=750,learning_rate=0.1,max_depth=3)
xgb_tuned.fit(X_train_red, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=750, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, tree_method='gpu_hist', verbosity=1)

In [0]:
predicted = xgb_tuned.predict(masterdf_test_red)

In [0]:
y_pred_rf = xgb_tuned.predict(X_test_red)
y_pred2_rf = xgb_tuned.predict_proba(X_test_red)

print(confusion_matrix(y_pred_rf, y_test))

print(classification_report(y_test, y_pred_rf))
y_pred2_rf = y_pred2_rf[:,1]
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_test, y_pred2_rf))

[[45113  4668]
 [  202   212]]
              precision    recall  f1-score   support

           0       0.91      1.00      0.95     45315
           1       0.51      0.04      0.08      4880

    accuracy                           0.90     50195
   macro avg       0.71      0.52      0.51     50195
weighted avg       0.87      0.90      0.86     50195

0.7625945498993385


In [0]:
predictions = pd.DataFrame(columns=['SK_ID_CURR', 'TARGET'])

In [0]:
predictions['SK_ID_CURR'] = masterdf_test0['SK_ID_CURR']

In [0]:
predictions['TARGET'] = predicted.tolist()

Neural Network

In [0]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import cross_val_score, train_test_split
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import RMSprop, Adadelta, Adam
from keras import regularizers
from keras import losses
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [0]:
y_train_Dummies = pd.get_dummies(y_train['TARGET'])


In [0]:
y_train_Dummies.head()

,0,1
279946,1,0
204052,0,1
110616,1,0
98521,1,0
247889,1,0


In [0]:
from keras import losses
from keras.layers import Dropout
model = Sequential()
model.add(Dense(60, input_dim=X_train_red.shape[1], activation='relu',kernel_regularizer=regularizers.l2(1)))
model.add(Dropout(0.3))
model.add(Dense(30, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_red, y_train_Dummies, epochs=100, batch_size=10000,  verbose=1)

Epoch 1/100
200778/200778 [==============================] - 1s 6us/step - loss: 442.8483 - acc: 0.5468
Epoch 2/100
200778/200778 [==============================] - 0s 2us/step - loss: 355.7884 - acc: 0.7977
Epoch 3/100
200778/200778 [==============================] - 0s 2us/step - loss: 277.2973 - acc: 0.8815
Epoch 4/100
200778/200778 [==============================] - 0s 2us/step - loss: 208.7040 - acc: 0.8954
Epoch 5/100
200778/200778 [==============================] - 0s 2us/step - loss: 150.1174 - acc: 0.8984
Epoch 6/100
200778/200778 [==============================] - 0s 2us/step - loss: 101.4803 - acc: 0.8996
Epoch 7/100
200778/200778 [==============================] - 0s 2us/step - loss: 62.2178 - acc: 0.9000
Epoch 8/100
200778/200778 [==============================] - 0s 2us/step - loss: 32.7628 - acc: 0.9004
Epoch 9/100
200778/200778 [==============================] - 0s 2us/step - loss: 13.1563 - acc: 0.9004
Epoch 10/100
200778/200778 [==============================] - 0s 2u

In [0]:
y_pred = model.predict(masterdf_test_red)
y_pred
# print('Mean Absolute Error:', metrics.mean_absolute_error(Ytest, y_pred))
# print('Mean Squared Error:', metrics.mean_squared_error(Ytest, y_pred))
# print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(Ytest, y_pred)))

array([[0.88958496, 0.11102748],
       [0.8895376 , 0.11107826],
       [0.889524  , 0.1110929 ],
       ...,
       [0.88950026, 0.11111808],
       [0.8895595 , 0.11105481],
       [0.88951015, 0.11110747]], dtype=float32)

In [0]:
y_pred[:,1]

array([0.11102748, 0.11107826, 0.1110929 , ..., 0.11111808, 0.11105481,
       0.11110747], dtype=float32)

In [0]:
prediction_nn = pd.DataFrame(columns=['SK_ID_CURR', 'TARGET'])

In [0]:
prediction_nn['SK_ID_CURR'] = masterdf_test0['SK_ID_CURR']

In [0]:
prediction_nn['TARGET'] = y_pred[:,1].tolist()